# Neuro-Fuzzy Computing - Project - Fall 2025
## Galaxy Zoo — Training Setup

This notebook trains and evaluates a neural network on the **training** portion of the Galaxy Zoo dataset:

- Images: `galaxy-zoo-the-galaxy-challenge/images_training_rev1/`
- Labels: `galaxy-zoo-the-galaxy-challenge/training_solutions_rev1.csv`

> **Note:** This dataset is a *37-output regression* problem (probabilities from a decision tree), not a single-label classifier.

Required file structure (Linux):
```
project_root/
├── NFC_Project.ipynb
└── galaxy-zoo-the-galaxy-challenge/
    ├── images_training_rev1/
    │   ├── 100008.jpg
    │   ├── 100023.jpg
    │   └── ...
    ├── training_solutions_rev1.csv
    └── resuts.csv
```
Notebook contents (high-level):
1. Load and validate the dataset (paths, CSV schema, image availability).
2. Build a PyTorch `Dataset` and `DataLoader` (with train/val/test split).
3. Define and train a CNN on the training split.
4. Evaluate on validation and held-out test splits (report regression metrics over 37 outputs).
5. Record training time and inference time, and run hyperparameter sensitivity experiments.

### Libraries

If any necesasry library is missing, create and run the following cell
```
%pip install numpy pandas pillow matplotlib scikit-learn torch torchvision --break-system-packages
```

### Checking PyTortch compatibility with CUDA
In this cell, we:
1. Choose the compute device this notebook should use. It is opted to run on an NVIDIA GPU.
2. Enable performance settings on our GPU.
    Specifically, we:
    - Let cuDNN search for the fastest convolution algorithms for our fixed input sizes (the supposed tradeoff is slightly less deterministic timing/behavior if input sizes vary, but we ensure the images are all sized 224x224).
    - Allow TF32 math on supported NVIDIA GPUs for matmul/conv, which can significantly speed up training with usually negligible accuracy impact.

In [1]:
import torch

print("torch:", torch.__version__)
print("torch built CUDA:", torch.version.cuda)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

if device.type == "cuda":
    # Performance knobs (safe defaults)
    torch.backends.cudnn.benchmark = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

    print("GPU:", torch.cuda.get_device_name(0))
    props = torch.cuda.get_device_properties(0)
    print(f"compute capability: {props.major}.{props.minor}")
    print(f"total VRAM (GiB): {props.total_memory / (1024**3):.2f}")

torch: 2.10.0+cu128
torch built CUDA: 12.8
device: cuda
GPU: NVIDIA GeForce RTX 5060 Laptop GPU
compute capability: 12.0
total VRAM (GiB): 7.53


### Dataset inspection and preprocessing

In the following cells we perform the essential preprocessing steps

1. **Define dataset paths and parameters**
   - Point to the raw training images folder: `images_training_rev1/`
   - Point to the label file: `training_solutions_rev1.csv`
   - Define an output folder for resized images: `images_training_224/`
   - Set the target image size to `(224, 224)`

In [2]:
from pathlib import Path
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision.transforms.functional import to_tensor
from PIL import Image

root = Path("galaxy-zoo-the-galaxy-challenge")
raw_img_dir = root / "images_training_rev1"
csv_path = root / "training_solutions_rev1.csv"
proc_img_dir = root / "images_training_224"
size = (224, 224)   # Size of each image inside "images_training_224"

2. **Verifiy required inputs exist**
   - Check that `images_training_rev1/` exists and contains `.jpg` files
   - Check that `training_solutions_rev1.csv` exists

In [3]:
if not raw_img_dir.is_dir():
    raise FileNotFoundError(f"Missing folder: {raw_img_dir.resolve()}")
if not csv_path.is_file():
    raise FileNotFoundError(f"Missing file: {csv_path.resolve()}")

proc_img_dir.mkdir(exist_ok=True)

raw_files = sorted([p for p in raw_img_dir.iterdir() if p.suffix.lower() == ".jpg"])
if not raw_files:
    raise FileNotFoundError(f"No .jpg files found in: {raw_img_dir.resolve()}")

3. **Offline preprocessing: resize images into a new folder**
   - Create `images_training_224/` if it doesn’t exist
   - If preprocessing is needed, we iterate through raw images and save a resized `(224, 224)` version into `images_training_224/`
   - Uses **LANCZOS** resampling for ensure high quality resizing

In [4]:
existing_proc = list(proc_img_dir.glob("*.jpg"))
if len(existing_proc) != len(raw_files):
    for p in raw_files:
        out = proc_img_dir / p.name
        if out.exists():
            continue
        img = Image.open(p).convert("RGB").resize(size, Image.Resampling.LANCZOS)
        img.save(out, format="JPEG", quality=95, optimize=True)

4. **Validate preprocessing results**
   - Open each processed image and confirms its size is exactly `(224, 224)`
   - Build a sorted list of processed training filenames for later consistency checks

In [5]:
for p in proc_img_dir.glob("*.jpg"):
    with Image.open(p) as img:
        if img.size != size:
            raise ValueError(f"Processed image has wrong size {img.size}, expected {size}: {p.name}")

train_image_names = sorted([p.name for p in proc_img_dir.glob("*.jpg")])

5. **Load and validate the label table**
   - Read `training_solutions_rev1.csv` into a DataFrame
   - Confirm a `GalaxyID` column exists
   - Confirm there are exactly **37 target columns** (Galaxy Zoo probability outputs)
   - Split labels into:
     - `ids` (GalaxyIDs, used to locate images on disk)
     - `labels` (a float32 NumPy array of shape `(N, 37)`)

In [6]:
labels_df = pd.read_csv(csv_path)
if "GalaxyID" not in labels_df.columns:
    raise ValueError("training_solutions_rev1.csv must contain a 'GalaxyID' column")

target_cols = [c for c in labels_df.columns if c != "GalaxyID"]
if len(target_cols) != 37:
    raise ValueError(f"Expected 37 target columns, found {len(target_cols)}")

labels_only = labels_df.drop(columns=["GalaxyID"])
labels = labels_only.astype("float32").to_numpy()
ids = labels_df["GalaxyID"].astype(str).tolist()

6. **Check label to image consistency**
   - Confirm the number of label rows matches the number of processed images
   - If there is a mismatch, we report example GalaxyIDs whose image files are missing

In [ ]:
if len(ids) != len(train_image_names):
    missing = []
    name_set = set(train_image_names)
    for gid in ids[:50]:
        if f"{gid}.jpg" not in name_set:
            missing.append(gid)
    raise ValueError(f"Label/image count mismatch: labels={len(ids)} images={len(train_image_names)}. Example missing IDs: {missing[:10]}")

7. **Build a PyTorch dataset (lazy loading)**
   - Define `GalaxyZooDataset`, which loads images **on demand** from `images_training_224/`
   - Return one sample as `(image_tensor, target_tensor, GalaxyID)` where:
     - `image_tensor` has shape `(3, 224, 224)` in `[0, 1]`
     - `target_tensor` has shape `(37,)`

In [8]:
class GalaxyZooDataset(Dataset):
    def __init__(self, img_dir: Path, ids, labels):
        self.img_dir = Path(img_dir)
        self.ids = list(ids)
        self.labels = labels

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        gid = self.ids[idx]
        img = Image.open(self.img_dir / f"{gid}.jpg").convert("RGB")
        x = to_tensor(img)  # float32, (C,H,W), range [0,1]

        # Normalize using ImageNet stats
        mean = torch.tensor((0.485, 0.456, 0.406)).view(3, 1, 1)
        std  = torch.tensor((0.229, 0.224, 0.225)).view(3, 1, 1)
        x = (x - mean) / std

        y = torch.from_numpy(self.labels[idx])
        return x, y, gid

8. **Run a smoke test**
   - Finally, read `dataset[0]` and print shapes + the GalaxyID to confirm everything is wired correctly

In [9]:
dataset = GalaxyZooDataset(proc_img_dir, ids, labels)

x, y, gid = dataset[0]
print(x.shape, y.shape, gid)

torch.Size([3, 224, 224]) torch.Size([37]) 100008


### Train/Validation/Test split + DataLoaders

Notes:
- Our train/validation/test split is **80/10/10** (80% training, 10% validation, 10% held-out test).
- We **shuffle the training data** (`shuffle=True`) so batches are not formed in any fixed file/ID order. This reduces bias from accidental ordering in the dataset and improves SGD/Adam convergence by making successive mini-batches more representative of the overall distribution.
- We train using **mini-batches** (size `BATCH_SIZE`) instead of loading all images at once. This is required for practical GPU training: it limits memory usage (VRAM) and enables efficient, stable gradient updates.
- We use multiple **DataLoader workers** (`NUM_WORKERS`) so image reading and preprocessing (disk I/O + JPEG decoding + tensor conversion) happen in parallel in background processes. This helps keep the GPU fed with data and reduces idle time between batches, especially when the CPU or disk is a bottleneck.
    - `pin_memory=True` can speed up CPU-to-GPU transfers by allocating page-locked host memory
    - `persistent_workers=True` keeps worker processes alive across epochs to reduce startup overhead (effective when `NUM_WORKERS > 0`).


In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset

indices = np.arange(len(dataset))
train_idx, temp_idx = train_test_split(indices, test_size=0.20, random_state=42, shuffle=True)
val_idx, test_idx   = train_test_split(temp_idx, test_size=0.50, random_state=42, shuffle=True)

train_ds = Subset(dataset, train_idx)
val_ds   = Subset(dataset, val_idx)
test_ds  = Subset(dataset, test_idx)

BATCH_SIZE = 32
NUM_WORKERS = 4

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True)
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True)

print("Sizes:", len(train_ds), len(val_ds), len(test_ds))

Sizes: 49262 6158 6158


### Our CNN models 
all having outputs of 37 values

In [11]:
import torch
import torch.nn as nn
import torchvision.models as tvm

class SimpleCNN37(nn.Module):
    def __init__(self, out_dim=37):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(256, out_dim)
        )

    def forward(self, x):
        return self.head(self.features(x))


class BetterCNN37(nn.Module):
    def __init__(self, out_dim=37):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1, bias=False), nn.BatchNorm2d(32), nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1, bias=False), nn.BatchNorm2d(32), nn.ReLU(),
            nn.MaxPool2d(2), nn.Dropout2d(0.05),

            nn.Conv2d(32, 64, 3, padding=1, bias=False), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1, bias=False), nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d(2), nn.Dropout2d(0.10),

            nn.Conv2d(64, 128, 3, padding=1, bias=False), nn.BatchNorm2d(128), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1, bias=False), nn.BatchNorm2d(128), nn.ReLU(),
            nn.MaxPool2d(2), nn.Dropout2d(0.15),

            nn.Conv2d(128, 256, 3, padding=1, bias=False), nn.BatchNorm2d(256), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1, bias=False), nn.BatchNorm2d(256), nn.ReLU(),
            nn.MaxPool2d(2), nn.Dropout2d(0.20)
        )
        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(256, out_dim)
        )

    def forward(self, x):
        return self.head(self.features(x))


def conv_bn_relu(in_ch, out_ch, k=3, p=1):
    return nn.Sequential(
        nn.Conv2d(in_ch, out_ch, kernel_size=k, padding=p, bias=False),
        nn.BatchNorm2d(out_ch),
        nn.ReLU(inplace=True),
    )


class InceptionBlock(nn.Module):
    def __init__(self, in_ch, b1, b3, b5, bp):
        super().__init__()
        self.branch1 = conv_bn_relu(in_ch, b1, k=1, p=0)
        self.branch3 = nn.Sequential(
            conv_bn_relu(in_ch, b3, k=1, p=0),
            conv_bn_relu(b3, b3, k=3, p=1),
        )
        self.branch5 = nn.Sequential(
            conv_bn_relu(in_ch, b5, k=1, p=0),
            conv_bn_relu(b5, b5, k=3, p=1),
            conv_bn_relu(b5, b5, k=3, p=1),
        )
        self.branch_pool = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            conv_bn_relu(in_ch, bp, k=1, p=0),
        )

    def forward(self, x):
        return torch.cat([self.branch1(x), self.branch3(x), self.branch5(x), self.branch_pool(x)], dim=1)


class InceptionCNN37(nn.Module):
    def __init__(self, out_dim=37):
        super().__init__()
        self.features = nn.Sequential(
            conv_bn_relu(3, 32), conv_bn_relu(32, 32), nn.MaxPool2d(2),
            conv_bn_relu(32, 64), conv_bn_relu(64, 64), nn.MaxPool2d(2),

            InceptionBlock(64, b1=32, b3=32, b5=32, bp=32),  # out: 128
            nn.MaxPool2d(2),

            conv_bn_relu(128, 256), conv_bn_relu(256, 256),
            nn.MaxPool2d(2),
        )
        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(256, out_dim)
        )

    def forward(self, x):
        return self.head(self.features(x))


AVAILABLE_MODELS = [
    "simplecnn",
    "bettercnn",
    "inceptioncnn",
    "resnet18",
    "efficientnet_b0",
    "efficientnet_b1",
    "densenet121",
    "vit_b_16",
    "cvt",
]


def make_model(name: str, out_dim: int = 37, pretrained: bool = True) -> nn.Module:
    name = name.lower().strip()

    if name == "simplecnn":
        return SimpleCNN37(out_dim)

    if name == "bettercnn":
        return BetterCNN37(out_dim)

    if name == "inceptioncnn":
        return InceptionCNN37(out_dim)

    if name == "resnet18":
        weights = tvm.ResNet18_Weights.DEFAULT if pretrained else None
        m = tvm.resnet18(weights=weights)
        m.fc = nn.Linear(m.fc.in_features, out_dim)
        return m

    if name == "efficientnet_b0":
        weights = tvm.EfficientNet_B0_Weights.DEFAULT if pretrained else None
        m = tvm.efficientnet_b0(weights=weights)
        m.classifier[1] = nn.Linear(m.classifier[1].in_features, out_dim)
        return m

    if name == "efficientnet_b1":
        weights = tvm.EfficientNet_B1_Weights.DEFAULT if pretrained else None
        m = tvm.efficientnet_b1(weights=weights)
        m.classifier[1] = nn.Linear(m.classifier[1].in_features, out_dim)
        return m

    if name == "densenet121":
        weights = tvm.DenseNet121_Weights.DEFAULT if pretrained else None
        m = tvm.densenet121(weights=weights)
        m.classifier = nn.Linear(m.classifier.in_features, out_dim)
        return m

    if name == "vit_b_16":
        weights = tvm.ViT_B_16_Weights.DEFAULT if pretrained else None
        m = tvm.vit_b_16(weights=weights)
        m.heads.head = nn.Linear(m.heads.head.in_features, out_dim)
        return m

    if name == "cvt":
        try:
            import timm
        except Exception as e:
            raise ImportError("Model 'cvt' requires the 'timm' package (pip install timm).") from e
        return timm.create_model("cvt-13", pretrained=pretrained, num_classes=out_dim)

    raise ValueError(
        f"Unknown model name: '{name}'. Available models: {', '.join(AVAILABLE_MODELS)}"
    )


MODEL_NAME = "simplecnn"
model = make_model(MODEL_NAME, out_dim=37, pretrained=True).to(device)
print(model)

SimpleCNN37(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (head): Sequential(
    (0): AdaptiveAvgPool2d(output_size=(1, 1))
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): Linear(in_features=256, out_features=37, bias=True)
  )
)


### Loss + optimizer

In [12]:
import torch.optim as optim

EPOCHS = 15
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-5)

@torch.no_grad()
def rmse_from_sse(sse: float, n: int) -> float:
    return (sse / n) ** 0.5

### Training loop
Now with early stopping

In [13]:
import time
import copy
import torch

def run_epoch(model, loader, train: bool):
    if train:
        model.train()
    else:
        model.eval()

    total_sse = 0.0
    total_n = 0
    total_loss = 0.0

    for x, y, _ in loader:
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        if train:
            optimizer.zero_grad(set_to_none=True)

        preds = model(x)
        preds = torch.sigmoid(preds)    # Adding sigmoid to constrain outputs between 0 and 1
        loss = criterion(preds, y)

        if train:
            loss.backward()
            optimizer.step()

        batch_sse = torch.sum((preds - y) ** 2).item()
        total_sse += batch_sse
        total_n += y.numel()
        total_loss += loss.item() * x.size(0)

    avg_mse = total_loss / len(loader.dataset)
    avg_rmse = rmse_from_sse(total_sse, total_n)
    return avg_mse, avg_rmse

patience = 2
min_delta = 1e-3  # require at least this much improvement

best_val = float("inf")
best_state = None
epochs_no_improve = 0

for epoch in range(1, EPOCHS + 1):
    t0 = time.time()
    train_mse, train_rmse = run_epoch(model, train_loader, train=True)
    val_mse, val_rmse     = run_epoch(model, val_loader,   train=False)
    dt = time.time() - t0

    improved = (best_val - val_rmse) > min_delta
    if improved:
        best_val = val_rmse
        best_state = copy.deepcopy(model.state_dict())
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    scheduler.step()

    print(
        f"Epoch {epoch:02d} | lr: {optimizer.param_groups[0]['lr']:.2e} | train RMSE {train_rmse:.6f} | val RMSE {val_rmse:.6f} "
        f"| best {best_val:.6f} | no_improve {epochs_no_improve}/{patience} | time {dt:.1f}s"
    )

    if epochs_no_improve >= patience:
        print("Early stopping triggered.")
        break

if best_state is not None:
    model.load_state_dict(best_state)
    print("Loaded best model weights from epoch with val RMSE:", best_val)

#EPOCHS = 10
#for epoch in range(1, EPOCHS + 1):
#    t0 = time.time()
#    train_mse, train_rmse = run_epoch(model, train_loader, train=True)
#    val_mse, val_rmse     = run_epoch(model, val_loader,   train=False)
#    dt = time.time() - t0
#    print(f"Epoch {epoch:02d} | train RMSE {train_rmse:.6f} | val RMSE {val_rmse:.6f} | time {dt:.1f}s")

Epoch 01 | lr: 9.89e-04 | train RMSE 0.156452 | val RMSE 0.148932 | best 0.148932 | no_improve 0/2 | time 103.3s
Epoch 02 | lr: 9.57e-04 | train RMSE 0.139484 | val RMSE 0.134007 | best 0.134007 | no_improve 0/2 | time 100.8s
Epoch 03 | lr: 9.05e-04 | train RMSE 0.128461 | val RMSE 0.127830 | best 0.127830 | no_improve 0/2 | time 100.6s
Epoch 04 | lr: 8.36e-04 | train RMSE 0.119848 | val RMSE 0.122059 | best 0.122059 | no_improve 0/2 | time 100.5s
Epoch 05 | lr: 7.52e-04 | train RMSE 0.114063 | val RMSE 0.111102 | best 0.111102 | no_improve 0/2 | time 100.2s
Epoch 06 | lr: 6.58e-04 | train RMSE 0.109975 | val RMSE 0.110199 | best 0.111102 | no_improve 1/2 | time 100.4s
Epoch 07 | lr: 5.57e-04 | train RMSE 0.106943 | val RMSE 0.106770 | best 0.106770 | no_improve 0/2 | time 100.5s
Epoch 08 | lr: 4.53e-04 | train RMSE 0.104284 | val RMSE 0.107549 | best 0.106770 | no_improve 1/2 | time 100.3s
Epoch 09 | lr: 3.52e-04 | train RMSE 0.102514 | val RMSE 0.103238 | best 0.103238 | no_improve 0

### Evaluate on held-out test split

In [14]:
test_mse, test_rmse = run_epoch(model, test_loader, train=False)
print("Test RMSE:", test_rmse)

Test RMSE: 0.10041120567714927


### Append to `results.csv`

In [15]:
import csv
from datetime import datetime
from pathlib import Path

test_mse, test_rmse = run_epoch(model, test_loader, train=False)
print("Test RMSE:", test_rmse)

timestamp = datetime.now().strftime("%m%d%H%M%S")
model_name = MODEL_NAME  # make sure you have this variable set
results_path = Path("results.csv")

file_exists = results_path.exists()

with results_path.open("a", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=["timestamp", "model", "RMSE"])
    if not file_exists:
        writer.writeheader()
    writer.writerow({
        "timestamp": timestamp,
        "model": str(model_name),
        "RMSE": float(test_rmse),
    })

print("Wrote to:", results_path.resolve())

Test RMSE: 0.10041120567714927
Wrote to: /home/user/Documents/Project/results.csv
